In [36]:
# %pip install torch torchvision torchaudio

In [37]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision.models import resnet50

In [38]:
INPUT_DIR_A = '../data/model_input_a'

In [39]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# Define the batch size
batch_size = 1

# Define the transformation
transform = transforms.Compose([
    transforms.RandomPerspective(),  # Apply a random perspective transformation
    transforms.ToTensor()  # Convert the image to a tensor
])

# Define the dataset
dataset = datasets.ImageFolder(INPUT_DIR_A, transform=transform)

# Verify the classes
classes = dataset.classes
print(classes)

dataset = DataLoader(dataset, batch_size=batch_size, shuffle=True)


['Monet', 'Picasso']


In [40]:
if torch.cuda.is_available():
    device = "cuda"
elif torch.backends.mps.is_available():
    device = "mps"
else:
    device = "cpu"

print(f"Device: {device}")

# Load the pre-trained ResNet model
model = resnet50(pretrained=True)

# Freeze all the parameters in the pre-trained model
for param in model.parameters():
    param.requires_grad = False

# Modify the last fully connected layer to match the number of classes in your dataset
num_classes = len(classes)
model.fc = nn.Linear(model.fc.in_features, num_classes)
model.to(device)

# Define the loss function
criterion = nn.CrossEntropyLoss()

# Define the optimizer
optimizer = optim.SGD(model.fc.parameters(), lr=0.001, momentum=0.9)

# Define the number of training epochs
num_epochs = 100

# Training loop
for epoch in range(num_epochs):
    running_loss = 0.0
    correct = 0
    total = 0
    
    # Set the model to training mode
    model.train()

    
    for images, labels in dataset:
        # Move the images and labels to the GPU if available
        images = images.to(device)
        labels = labels.to(device)
        
        # Zero the gradients
        optimizer.zero_grad()
        
        # Forward pass
        outputs = model(images)
        
        # Compute the loss
        loss = criterion(outputs, labels)
        
        # Backward pass
        loss.backward()
        
        # Update the weights
        optimizer.step()
        
        # Update the running loss
        running_loss += loss.item()
        
        # Compute the predicted labels
        _, predicted = torch.max(outputs.data, 1)
        
        # Update the total and correct predictions
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    
    # Compute the accuracy
    accuracy = 100 * correct / total
    
    # Print the loss and accuracy for each epoch
    print(f"Epoch {epoch+1}/{num_epochs}: Loss = {running_loss:.4f}, Accuracy = {accuracy:.2f}%")
    if accuracy > 90:
        break


Device: mps
Epoch 1/100: Loss = 22.3162, Accuracy = 52.63%
Epoch 2/100: Loss = 15.8583, Accuracy = 52.63%
Epoch 3/100: Loss = 16.9821, Accuracy = 36.84%
Epoch 4/100: Loss = 16.6657, Accuracy = 47.37%
Epoch 5/100: Loss = 17.7350, Accuracy = 42.11%
Epoch 6/100: Loss = 29.8824, Accuracy = 36.84%
Epoch 7/100: Loss = 15.2798, Accuracy = 36.84%
Epoch 8/100: Loss = 16.6746, Accuracy = 42.11%
Epoch 9/100: Loss = 18.4215, Accuracy = 57.89%
Epoch 10/100: Loss = 15.6151, Accuracy = 36.84%
Epoch 11/100: Loss = 16.5076, Accuracy = 57.89%
Epoch 12/100: Loss = 27.5730, Accuracy = 52.63%
Epoch 13/100: Loss = 12.3275, Accuracy = 63.16%
Epoch 14/100: Loss = 28.5270, Accuracy = 57.89%
Epoch 15/100: Loss = 15.5062, Accuracy = 36.84%
Epoch 16/100: Loss = 19.6467, Accuracy = 36.84%
Epoch 17/100: Loss = 23.3466, Accuracy = 52.63%
Epoch 18/100: Loss = 23.9184, Accuracy = 42.11%
Epoch 19/100: Loss = 22.1657, Accuracy = 15.79%
Epoch 20/100: Loss = 22.9664, Accuracy = 42.11%
Epoch 21/100: Loss = 13.9528, Accurac